### Installing Dependencies

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.2 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

### Load Summarization Pipeline

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


### Summarize Text by manual copy paste

In [4]:
ARTICLE = """
You don’t always have to give your boss the finger
Maybe it’s your first day on the job. Perhaps your manager just made an announcement. You’ve been asked to scan your fingerprint every time you clock in and out. Is that even allowed?
From Hooters to Hyatt Hotels, employers tantalized by the promise of a futuristic, streamlined way to track workers’ attendance are starting to use time clock machines that fingerprint employees.
Vendors like Kronos and Allied Time say that because the machines are tied to your biometric information — unique characteristics such as your face, fingerprints, how you talk, and even how you walk — they provide a higher level of workplace security and limit employees’ ability to commit “time theft” by punching in for one another.
But the benefits for your boss may come at a cost to you — both your privacy and possibly your health.
With the global outbreak of COVID-19, your personal health could be at risk when using frequently touched screens and fingerprint scanners. The Centers for Disease Control says that coronavirus can remain on surfaces for hours, so screens and scanners should be regularly disinfected with cleaning spray or wipes. And you should wash your hands for 20 seconds or use alcohol-based hand sanitizer immediately after using one.
In addition to these health concerns, critics argue that biometric devices pose massive personal security issues, exposing workers to potential identity theft and subjecting them to possible surveillance from corporations and law enforcement.
In an amicus brief in a case before a federal court of appeals, a group of privacy advocates, including the ACLU and the EFF, wrote that “the immutability of biometric information” puts people “at risk of irreparable harm in the form of identity theft and/or tracking.”
“You can get a new phone, you can change your password, you can even change your Social Security number; you can’t change your face,” said Kade Crockford, the Technology for Liberty program director at ACLU of Massachusetts.
Companies facing legal action over their use of the machines range from fast food joints like McDonald’s and Wendy’s, to hotel chains like Marriott and Hyatt, to airlines like United and Southwest.
In some cases, the companies have countered in the lawsuits that their employees’ union agreement allows the use of the machines: “Southwest and United contend that the plaintiffs’ unions have consented — either expressly or through the collective bargaining agreements’ management-rights clauses — and that any required notice has been provided to the unions,” the court’s opinion states.
Other companies have not responded to requests for comment or have said they cannot comment on active litigation.
Privacy and labor laws have lagged behind the shifts in the American workplace. But in some places, you have the right to refuse and even sue.

Biometric Privacy Laws
As the collection and use of biometrics has exploded, lawmakers in three states have responded by passing laws restricting its deployment.
"""

In [5]:
summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)

[{'summary_text': ' Employers are starting to use time clock machines that fingerprint employees . The machines are tied to your unique characteristics such as your face, fingerprints, how you talk, and even how you walk . The Centers for Disease Control says that coronavirus can remain on surfaces for hours .'}]

### Summarize Text reading automatically from a blog post

In [6]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
#URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [7]:
r = requests.get(URL)
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [8]:
ARTICLE

'Sign up Sign In Sign up Sign In Towards Data Science Feb 1, 2021 Member-only Save A Bayesian Take On Model Regularization I’m currently reading “How We Learn” by Stanislas Dehaene. First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI. One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i.e. given only a limited amount of experience¹. One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world. In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models. But why do we do this, even from birth¹? One argument is that, contrary to the frequentist view in child psychology (the belief that babies learn solely through th

### Chunk Text (since hugging face summerizer will take only 512 tokens at once)

In [9]:
max_chunk = 500

In [10]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [11]:
sentences = ARTICLE.split('<eos>')
chunks = []
temp = []
current_chunk_length= 0

for i in range(len(sentences)):
  if ( ( current_chunk_length + len(sentences[i].split(' ')) ) < max_chunk):
    current_chunk_length +=  len(sentences[i].split(' '))
    temp.extend(sentences[i].split(' '))
  else :
    chunks.append(temp)
    temp = []
    temp.extend(sentences[i].split(' '))
    current_chunk_length =  len(sentences[i].split(' '))



# for last sentence (check if it is not empty then append)
if temp:
  chunks.append(temp)

# join the tokens back into sentences
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [12]:
'''sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []


for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])'''

"sentences = ARTICLE.split('<eos>')\ncurrent_chunk = 0 \nchunks = []\n\n\nfor sentence in sentences:\n    if len(chunks) == current_chunk + 1: \n        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:\n            chunks[current_chunk].extend(sentence.split(' '))\n        else:\n            current_chunk += 1\n            chunks.append(sentence.split(' '))\n    else:\n        print(current_chunk)\n        chunks.append(sentence.split(' '))\n\nfor chunk_id in range(len(chunks)):\n    chunks[chunk_id] = ' '.join(chunks[chunk_id])"

In [13]:
for i in range(len(chunks)):
  print(len(chunks[i].split(' ')))

496
491
498
335


In [14]:
chunks[0]

'Sign up Sign In Sign up Sign In Towards Data Science Feb 1, 2021 Member-only Save A Bayesian Take On Model Regularization I’m currently reading “How We Learn” by Stanislas Dehaene.  First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI.  One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i. e.  given only a limited amount of experience¹.  One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world.  In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models.  But why do we do this, even from birth¹?  One argument is that, contrary to the frequentist view in child psychology (the belief that babies learn solely th

### Summarize Text

In [15]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [16]:
res[0]

{'summary_text': ' In machine learning, regularization, or model complexity control, is an essential and common practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models . We will focus on analytical regularization techniques, since their Bayesian interpretation is more well-defined .'}

In [17]:
' '.join([summ['summary_text'] for summ in res])

' In machine learning, regularization, or model complexity control, is an essential and common practice to ensure that a model attains high out-of-sample performance . We can do this by placing a prior on our distribution of models . We will focus on analytical regularization techniques, since their Bayesian interpretation is more well-defined .  We will analyze these claims for regression problems, but they extend to other supervised learning tasks, such as classification, as well . We will focus on rigorously presenting the mathematics behind these claims . In the next section, we will use Bayes’ Rule to derive our L2 regularized estimator .  Using Bayes’ Rule, we can show that the mean and mode of the posterior distribution of w is the solution for LASSO regression when we invoke a Gaussian prior distribution on w . We’ll now examine a similar case with a Laplace prior . Here is the corresponding derivation for Lasso: (Note that in the last step, we set p = 1, q = 1)  A huge thank y

In [18]:
text = ' '.join([summ['summary_text'] for summ in res])

### save to a file

In [19]:
with open('blogsummary_by_BERT.txt', 'w') as f:
    f.write(text)

### END

### This is referred from Nick Youtube video partly.


## TODO setup this to run in conda env. Add URL, BeautifulSOUP html tags p,h1, and take care of output saving/displaying   as argument.